In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
import random
import re
import requests
import json

In [2]:
# Set the executable path and initialize the chrome browser 
# ----------------------MAC-----------------------------------------
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# # ======================Windows=====================================
# executable_path = {'executable_path': './chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Store start date, end date, source, and destination as variables
startDate = "8/24/2019"
endDate = "9/1/2019"
source = "SFO" # THE 3 IN THE BAY
destination = "SYD" # THE TOP DESTINATIONS OF 2019
url = 'https://www.expedia.com/Flights-Search?flight-type=on&starDate={0}&endDate={1}&mode=search&trip=roundtrip&leg1=from%3A{2}to%3A{3}%2Cdeparture%3A05%2F04%2F2019TANYT&leg2=from%3A{3}%2Cto{2}%2Cdeparture%3A05%2F18%2F2019TANYT&passengers=children%3A0%2Cadults%3A1%2Cseniors%3A0%2Cinfantinlap%3AY'.format(startDate, endDate, source, destination)

browser.visit(url)

In [4]:
# Simulate the entering of the dates into the browser to search
browser.find_by_id("departure-date-1").fill(startDate)
browser.find_by_id("return-date-1").fill(endDate)
browser.find_by_id("flight-wizard-search-button").click()

In [5]:
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}

In [6]:
response = requests.get(browser.url,headers = header)

In [7]:
# Use regex parse through and store the current session key
key = re.search("(?<=Flight-Search-Paging\?c\=).+(?=&is)",response.text).group(0)

In [8]:
# Create a dynamic API URL using the session key stored above
api = f'https://www.expedia.com/Flight-Search-Paging?c={key}&is=1&sp=asc&cz=200&cn=0&ul=0'
api

'https://www.expedia.com/Flight-Search-Paging?c=f87781ef-efd1-420f-ba74-652aa7fdfb52&is=1&sp=asc&cz=200&cn=0&ul=0'

In [261]:
response2 = requests.get(api, headers = header)

In [262]:
# Store the response into a variable
data = response2.json()
# print(data)

In [263]:
info = data['content']['legs']
offers = data['content']['offers']
# print(info)

In [264]:
# Pull flight leg IDs, flight duration, number of stops, departure airport, departure city, arrival airport
# ...arrival city, and price

leg_list = []

for key,value in info.items():
    leg_dict = {}
    leg_dict["leg_id"] = value['identity']['naturalKey']
    leg_dict["duration_hrs"] = value['timeline'][0]['duration']['hours']
    leg_dict["duration_mins"] = value['timeline'][0]['duration']['minutes']
    leg_dict["stops"] = value['stops']
    leg_dict["dept_airport"] = value['departureLocation']['airportCode']
    leg_dict["dept_city"] = value['departureLocation']['airportCity']
    leg_dict["arr_airport"] = value['arrivalLocation']['airportCode']
    leg_dict["arr_city"] = value['arrivalLocation']['airportCity']
    leg_dict["price"] = value['price']['offerPrice']
    
    legs_hours = []
    legs_mins = []
    for item in value["timeline"]:
        if item["layover"] == False:
            legs_hours.append(item["duration"]["hours"])
            legs_mins.append(item["duration"]["minutes"])

    leg_dict["layover_hrs"] = legs_hours
    leg_dict["layover_mins"] = legs_mins
    
    leg_list.append(leg_dict)

print(leg_list)

[{'leg_id': '29954cc3016c9c3d260c525d1b4559a6', 'duration_hrs': 5, 'duration_mins': 22, 'stops': 1, 'dept_airport': 'SFO', 'dept_city': 'San Francisco', 'arr_airport': 'YUL', 'arr_city': 'Montreal', 'price': 536.0, 'layover_hrs': [5, 1], 'layover_mins': [22, 43]}, {'leg_id': 'bd20fa7ddc19a212d5e8c4ce2d09ea39', 'duration_hrs': 2, 'duration_mins': 38, 'stops': 1, 'dept_airport': 'SFO', 'dept_city': 'San Francisco', 'arr_airport': 'YUL', 'arr_city': 'Montreal', 'price': 508.0, 'layover_hrs': [2, 4], 'layover_mins': [38, 3]}, {'leg_id': 'bd574f62c1de2778b96fb66dfbb9488b', 'duration_hrs': 1, 'duration_mins': 33, 'stops': 2, 'dept_airport': 'YUL', 'dept_city': 'Montreal', 'arr_airport': 'SFO', 'arr_city': 'San Francisco', 'price': 564.0, 'layover_hrs': [1, 5, 2], 'layover_mins': [33, 10, 20]}, {'leg_id': '5a25ce241e62b59423eeceb3b2f8a2ec', 'duration_hrs': 5, 'duration_mins': 28, 'stops': 0, 'dept_airport': 'SFO', 'dept_city': 'San Francisco', 'arr_airport': 'YUL', 'arr_city': 'Montreal', 'pr

In [265]:
leg_list_df = pd.DataFrame(leg_list)
leg_list_df.head(10)

,arr_airport,arr_city,dept_airport,dept_city,duration_hrs,duration_mins,layover_hrs,layover_mins,leg_id,price,stops
0,YUL,Montreal,SFO,San Francisco,5,22,"[5, 1]","[22, 43]",29954cc3016c9c3d260c525d1b4559a6,536.0,1
1,YUL,Montreal,SFO,San Francisco,2,38,"[2, 4]","[38, 3]",bd20fa7ddc19a212d5e8c4ce2d09ea39,508.0,1
2,SFO,San Francisco,YUL,Montreal,1,33,"[1, 5, 2]","[33, 10, 20]",bd574f62c1de2778b96fb66dfbb9488b,564.0,2
3,YUL,Montreal,SFO,San Francisco,5,28,[5],[28],5a25ce241e62b59423eeceb3b2f8a2ec,659.0,0
4,YUL,Montreal,SFO,San Francisco,4,31,"[4, 1]","[31, 45]",683f7f721914a4d0b2781f25d918ac4d,591.0,1
5,YUL,Montreal,SFO,San Francisco,4,8,"[4, 2]","[8, 6]",f867ef930e8cfa4a1d8cf8a885ceac62,817.0,1
6,YUL,Montreal,SFO,San Francisco,2,18,"[2, 1, 4]","[18, 24, 7]",af3147e7030d5830ed1ea2d5d2af2344,564.0,2
7,YUL,Montreal,SFO,San Francisco,5,16,"[5, 1]","[16, 15]",f15baa1852c1b1d10d4375aab2f04d8d,566.0,1
8,YUL,Montreal,SFO,San Francisco,5,28,[5],[28],eef66cb55f35d144f2b61d5a208549c7,4680.0,0
9,YUL,Montreal,SFO,San Francisco,4,55,"[4, 1]","[55, 15]",d17817901cc33fe20e5baba9dd9624c6,511.0,1


In [266]:
# Sum up hours and mins, convert all hours to mins

def flatten(data):
    total = []
    for x in list(data):
        total.append(sum(x))
    print(total)
    return total

# dataframe column only
total_hrs = flatten(leg_list_df['layover_hrs'])
total_mins = flatten(leg_list_df['layover_mins'])

[6, 6, 8, 5, 5, 6, 7, 6, 5, 5, 6, 5, 6, 7, 7, 6, 5, 6, 5, 5, 6, 6, 6, 5, 6, 7, 5, 6, 6, 6, 7, 6, 7, 5, 6, 5, 6, 6, 5, 6, 6, 6, 5, 6, 6, 5, 5, 6, 6, 5, 6, 5, 6]
[65, 41, 63, 28, 76, 14, 49, 31, 28, 70, 83, 70, 16, 97, 42, 60, 28, 83, 28, 71, 60, 69, 64, 85, 61, 47, 88, 52, 92, 77, 67, 31, 44, 90, 46, 28, 79, 13, 71, 102, 86, 20, 88, 63, 47, 80, 71, 41, 72, 28, 16, 28, 52]


In [267]:
# Add in new columns to store total hours and total mins
leg_list_df['total_hrs'] = total_hrs
leg_list_df['total_mins'] = total_mins
leg_list_df.head()

,arr_airport,arr_city,dept_airport,dept_city,duration_hrs,duration_mins,layover_hrs,layover_mins,leg_id,price,stops,total_hrs,total_mins
0,YUL,Montreal,SFO,San Francisco,5,22,"[5, 1]","[22, 43]",29954cc3016c9c3d260c525d1b4559a6,536.0,1,6,65
1,YUL,Montreal,SFO,San Francisco,2,38,"[2, 4]","[38, 3]",bd20fa7ddc19a212d5e8c4ce2d09ea39,508.0,1,6,41
2,SFO,San Francisco,YUL,Montreal,1,33,"[1, 5, 2]","[33, 10, 20]",bd574f62c1de2778b96fb66dfbb9488b,564.0,2,8,63
3,YUL,Montreal,SFO,San Francisco,5,28,[5],[28],5a25ce241e62b59423eeceb3b2f8a2ec,659.0,0,5,28
4,YUL,Montreal,SFO,San Francisco,4,31,"[4, 1]","[31, 45]",683f7f721914a4d0b2781f25d918ac4d,591.0,1,5,76


In [268]:
# Convert hours to mins, then add hours and mins together for the total time

leg_list_df["total_mins_final"]  = leg_list_df["total_hrs"] * 60
leg_list_df["total_time_mins"] =  leg_list_df["total_mins_final"] + leg_list_df["total_mins"]
leg_list_df["timestamp"] = time.time()
leg_list_df.to_csv('./data/leg_list_20190605_YUL.csv')
leg_list_df.head()

,arr_airport,arr_city,dept_airport,dept_city,duration_hrs,duration_mins,layover_hrs,layover_mins,leg_id,price,stops,total_hrs,total_mins,total_mins_final,total_time_mins,timestamp
0,YUL,Montreal,SFO,San Francisco,5,22,"[5, 1]","[22, 43]",29954cc3016c9c3d260c525d1b4559a6,536.0,1,6,65,360,425,1.559793e+09
1,YUL,Montreal,SFO,San Francisco,2,38,"[2, 4]","[38, 3]",bd20fa7ddc19a212d5e8c4ce2d09ea39,508.0,1,6,41,360,401,1.559793e+09
2,SFO,San Francisco,YUL,Montreal,1,33,"[1, 5, 2]","[33, 10, 20]",bd574f62c1de2778b96fb66dfbb9488b,564.0,2,8,63,480,543,1.559793e+09
3,YUL,Montreal,SFO,San Francisco,5,28,[5],[28],5a25ce241e62b59423eeceb3b2f8a2ec,659.0,0,5,28,300,328,1.559793e+09
4,YUL,Montreal,SFO,San Francisco,4,31,"[4, 1]","[31, 45]",683f7f721914a4d0b2781f25d918ac4d,591.0,1,5,76,300,376,1.559793e+09


In [269]:
# Pull second table of Offer ID, leg 1 and leg 2, and price

offer_list = []

for key,value in offers.items():
    offer_dict = {}
    offer_dict["offer_id_dept"] = value['legIds'][0]
    offer_dict["offer_id_return"] = value['legIds'][1]
    offer_dict["price"] = value['price']['exactPrice']
    offer_list.append(offer_dict)

print(offer_list)

[{'offer_id_dept': '82fc6d1dd067fbd0300145709ed63875', 'offer_id_return': '72bc2fb39a4f8b29eeeccd052d404faa', 'price': 591.0}, {'offer_id_dept': '3c59b9ab48280178af9cff50b1ab5fa8', 'offer_id_return': '1ec9540f91a938984d99bef1225ab301', 'price': 630.77}, {'offer_id_dept': '500bd6cb4dda921d67e0ea88b6b9686e', 'offer_id_return': '72bc2fb39a4f8b29eeeccd052d404faa', 'price': 1206.2}, {'offer_id_dept': 'af3147e7030d5830ed1ea2d5d2af2344', 'offer_id_return': 'bd574f62c1de2778b96fb66dfbb9488b', 'price': 563.46}, {'offer_id_dept': '32944e626c8b066d7213b3db99bec4a0', 'offer_id_return': '72bc2fb39a4f8b29eeeccd052d404faa', 'price': 428.81}, {'offer_id_dept': '550e0e694de984c1eee48e92847c569a', 'offer_id_return': '580d91093bb93833ae182bf8dc8c0313', 'price': 694.72}, {'offer_id_dept': '9ac054718f737d2735d47bdca1fbdb23', 'offer_id_return': '6d6287473682c247d109f136bcf3f6e4', 'price': 889.1600000000001}, {'offer_id_dept': '9684aacf40e747fc7fafbdb44c86bb87', 'offer_id_return': 'e623ebb9edf29f957b3f39e2e2

In [270]:
offer_list_df = pd.DataFrame(offer_list)
offer_list_df.to_csv('./data/offer_list_20190605_YUL.csv')
offer_list_df.head()

,offer_id_dept,offer_id_return,price
0,82fc6d1dd067fbd0300145709ed63875,72bc2fb39a4f8b29eeeccd052d404faa,591.00
1,3c59b9ab48280178af9cff50b1ab5fa8,1ec9540f91a938984d99bef1225ab301,630.77
2,500bd6cb4dda921d67e0ea88b6b9686e,72bc2fb39a4f8b29eeeccd052d404faa,1206.20
3,af3147e7030d5830ed1ea2d5d2af2344,bd574f62c1de2778b96fb66dfbb9488b,563.46
4,32944e626c8b066d7213b3db99bec4a0,72bc2fb39a4f8b29eeeccd052d404faa,428.81


In [ ]:
# Import my csvs and convert them all to dataframes
df1 = pd.read_csv('./data/leg_list_20190605_SYD.csv')
df2 = pd.read_csv('./data/leg_list_20190605_FCO.csv')
df3 = pd.read_csv('./data/leg_list_20190605_CUZ.csv')
df4 = pd.read_csv('./data/leg_list_20190605_BOS.csv')
df5 = pd.read_csv('./data/leg_list_20190605_YUL.csv')
df6 = pd.read_csv('./data/leg_list_20190605_NAN.csv')
df7 = pd.read_csv('./data/leg_list_20190605_NRT.csv')
df8 = pd.read_csv('./data/leg_list_20190605_IST.csv')
df9 = pd.read_csv('./data/leg_list_20190605_ZRH.csv')
df10 = pd.read_csv('./data/leg_list_20190605_BCN.csv')
df11 = pd.read_csv('./data/leg_list_20190605_GIG.csv')
df12 = pd.read_csv('./data/leg_list_20190605_CPT.csv')

In [ ]:
# Combine all the dataframes into 1 file
df_20190605 = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12])
# # df_20190603_v2 = df_20190603.drop(columns=['Unnamed: 0'])
df_20190605.to_csv('./data/df_20190605.csv')
len(df_20190605)

In [ ]:
# Import Google Maps API to find lat lng coordinates for cities
import googlemaps
from datetime import datetime

In [ ]:
# Read in csv with data and store as a dataframe
most_df = pd.read_csv('./data/df_20190605.csv')
most_df = most_df[['arr_city', 'dept_city', 'leg_id', 'price', 'stops', 'total_time_mins']]
most_df.head()

In [ ]:
gmaps = googlemaps.Client(key='API_Key')

In [ ]:
# Store all airports in a list, in a variable
airports = ['San Francisco', 'Sydney', 'Cusco', 'Rome', 'Rio de Janiero', 'Montreal', 'Boston', 'Cape Town', 'Istanbul', 'Nadi', 'Zurich', 'Barcelona']
arr_city = list(most_df['arr_city'])
dept_city = list(most_df['dept_city'])

In [ ]:
# Create a function to loop through cities and return lat lng coordinates
def get_coords(address):
    coordinates = gmaps.geocode(address)[0]['geometry']['location']
    coordinates["city"] = address
    coordinates["coordinates"] = (coordinates['lat'],coordinates['lng'])
    return(coordinates)

In [ ]:
# Store arrival city coordinates into a list
arr_city_results = []

for city in arr_city:
    arr_city_results.append(get_coords(city))
print(arr_city_results )

In [ ]:
# Store departure city coordinates into a list
dept_city_results = []

for city in dept_city:
    dept_city_results.append(get_coords(city))
print(dept_city_results)

In [ ]:
# Add columns to most_df
arr_city_result = [ (arr_city["lat"], arr_city["lng"]) for arr_city in arr_city_results]
dept_city_result = [ (dept_city["lat"], dept_city["lng"]) for dept_city in dept_city_results]


most_df["arr_city_coords"] = arr_city_result
most_df["dept_city_coords"] = dept_city_result
most_df
most_df.to_csv('./data/most_df_0605.csv')
# most_df.head()

In [ ]:
# Import geopy to calculate distance between the departure city and arrival city coordinates
import geopy.distance

dept_coords = list(most_df['dept_city_coords'])
arr_coords= list(most_df['arr_city_coords'])
distance = []

for index in range(len(most_df['dept_city_coords'])):
    distance.append(geopy.distance.distance(dept_coords[index], arr_coords[index]).miles)

SFO_coords = (37.6213129, -122.3789554)
SYD_coords = (-33.8688197, 151.2092955)

most_df["distance"] = distance
most_df.to_csv('./data/final_data_0605.csv')
most_df